In [ ]:
import numpy as np
import training_data_generation as tdg
import matplotlib.pyplot as plt
import matplotlib as mpl

%matplotlib inline
plt.ion();

#plt.style.use('classic')
fontsize=22
params = {
  'font.size' : fontsize,
  #'axes.titlesize' : fontsize,
  #'axes.labelsize': fontsize,
  #'legend.fontsize': fontsize,
  #'xtick.labelsize': fontsize,
  #'ytick.labelsize': fontsize,
  'text.usetex': True,
  'axes.unicode_minus': True}
plt.rcParams.update(params)
plt.rc('font', **{'family':'serif', 'serif':['Computer Modern Roman'],
                        'monospace': ['Computer Modern Typewriter']})

In [ ]:
import healpy as hp
import utils_intensity_map as uim
import utils_deck_generation as idg
import netcdf_read_write as nrw
import training_data_generation as tdg
import utils_healpy as uhp
import os
%matplotlib inline
np_complex = np.vectorize(complex)

In [ ]:
num_comparisons = 2

diag_dir = [None] * num_comparisons
iex = [None] * num_comparisons
sys_params = [None] * num_comparisons
run_location = [None] * num_comparisons

In [ ]:
plot_labels = ["N190204_003","Optimized Config"]#["N19","New"]#
modes_aspect_ratio = (7.2,3) #(3,6)
max_mode = 30
modes_xlabel = "l mode"
modes_ylabel = r"Mode amplitude ($\%$)"

In [ ]:
diag_dir[0] = "Data_230802a_N190204_003_4ns_2_3_weighting"
iex[0] = 0
sys_params[0] = tdg.define_system_params(diag_dir[0])
run_location[0] = sys_params[0]["root_dir"] + "/" + sys_params[0]["sim_dir"] + str(iex[0])

diag_dir[1] = "Data_230802a_N190204_003_4ns_2_3_weighting"
iex[1] = "517_scaled_new"
sys_params[1] = tdg.define_system_params(diag_dir[1])
run_location[1] = sys_params[1]["root_dir"] + "/" + sys_params[1]["sim_dir"] + str(iex[1])

with_pointing_markers = True
import_flipped = False
old_format = True
display_steradians = False

## Initial intensity modes

In [ ]:
dataset_params = [None] * num_comparisons
facility_spec = [None] * num_comparisons
intensity_map_sr = [None] * num_comparisons
intensity_map = [None] * num_comparisons
parameters = [None] * num_comparisons
intensity_map_normalized = [None] * num_comparisons
avg_flux = [None] * num_comparisons
rms1 = [None] * num_comparisons
power_spectrum_unweighted = [None] * num_comparisons
total_TW_init = [None] * num_comparisons


In [ ]:
for ind in range(num_comparisons):
    num_examples = 1
    sys_params[ind]["run_gen_deck"] = False
    print(plot_labels[ind])

    if old_format:
        dataset_params[ind], facility_spec[ind] = tdg.define_dataset_params(int(num_examples))
    else:
        dataset_params[ind] = nrw.read_general_netcdf(diag_dir[ind] + "/" + sys_params[ind]["dataset_params_filename"])
        facility_spec[ind] = nrw.read_general_netcdf(diag_dir[ind] + "/" + sys_params[ind]["facility_spec_filename"])
    dataset_params[ind]["num_examples"] = num_examples

    if old_format:
        intensity_map_sr[ind] = nrw.read_intensity(run_location[ind], dataset_params[ind]["imap_nside"], facility_spec[ind]['target_radius'])
        # convert from W/sr to W/cm**2
        intensity_map[ind] = intensity_map_sr[ind] / (facility_spec[ind]['target_radius'] / 10000.0)**2
    else:
        parameters[ind] = nrw.read_general_netcdf(run_location[ind] + "/" + sys_params[ind]["ifriit_ouput_name"])
        intensity_map[ind] = parameters[ind]["intensity"]
        # convert from W/cm**2 to W/sr
        intensity_map_sr[ind] = parameters[ind]["intensity"] * (facility_spec[ind]['target_radius'] / 10000.0)**2

    intensity_map_normalized[ind], avg_flux[ind] = uim.imap_norm(intensity_map[ind])
    rms1[ind] = uim.extract_rms(intensity_map_normalized[ind])
    
    LMAX = dataset_params[ind]["LMAX"]
    power_spectrum_unweighted[ind] = uhp.power_spectrum(intensity_map[ind], LMAX, verbose=False)

    avg_intensity_sr = np.mean(intensity_map_sr[ind])
    total_TW_init[ind] = avg_intensity_sr*10**(-12) * 4.0 * np.pi
    mean_intensity_cm = avg_intensity_sr / (facility_spec[ind]['target_radius'] / 10000.0)**2
    
    print('Mean intensity, {:.2e}W/cm2'.format(mean_intensity_cm))
    print('Intensity per steradian, {:.2e}W/sr'.format(avg_intensity_sr))
    print('The total power deposited is {:.2f}TW, '.format(total_TW_init[ind]))
    

In [ ]:
aspc = 1.2
fig1 = plt.figure(num=1, figsize=(6*aspc,6), facecolor='white')

nside = 64
npix = hp.nside2npix(nside)
#vec = hp.ang2vec(np.pi, np.pi * 3 / 4)
vec = hp.ang2vec(np.pi/2, np.pi * 1/2)
ipix_disc = hp.query_disc(nside=nside, vec=vec, radius=np.radians(90))
masked_intensity_map = intensity_map[0]
masked_intensity_map[ipix_disc] = intensity_map[1][ipix_disc]

projection = hp.mollview(masked_intensity_map, unit=r"$\rm{W/cm^2}$", title="", flip="geo", return_projected_map=True)
hp.graticule()

In [ ]:
aspc = 2.0
fig2 = plt.figure(num=2, figsize=(6*aspc,6), facecolor='white')
ax2 = fig2.add_axes([0.15, 0.14, 0.6, 0.6])
cax1 = fig2.add_axes([0.76, 0.14, 0.02, 0.6])

cmesh = ax2.pcolormesh(projection, linewidth=0, rasterized=True)
cbar = fig2.colorbar(cmesh, ax=[ax2], cax=cax1)
ax2.plot([400.5,400.5],[0,400.0],"k", linewidth=1.0)
ax2.axis('off')

#ax2.annotate(plot_labels[0], xy=(200.5,200.5), fontsize=22, color='black', fontweight="bold")
#ax2.text(190.5,410.5, plot_labels[0], fontsize=22, color='black', fontweight='bold')
#ax2.text(420.5,410.5, plot_labels[1], fontsize=22, color='black', fontweight='bold')

cbar.set_label("Intensity" + r" $(W/cm^2)$")
fig2.savefig(sys_params[0]["figure_location"]+"/init_mollweide" + sys_params[0]["plot_file_type"], dpi=300, bbox_inches='tight')


In [ ]:
#aspc = 1.2
#fig1 = plt.figure(num=1, figsize=(6*aspc,3), facecolor='white')
fig1 = plt.figure(num=1, figsize=modes_aspect_ratio, facecolor='white')
ax1 = fig1.add_axes([0.15, 0.15, 0.8, 0.8])

for ind in range(num_comparisons):
    print(plot_labels[ind])
    ax1.plot(np.arange(LMAX), power_spectrum_unweighted[ind] * 100.0, linewidth=3)
    rms1[ind] = np.sqrt(np.sum(np.array(power_spectrum_unweighted[ind])**2))
    print('RMS is {:.2f}%, '.format(rms1[ind] * 100.0))

ax1.set_xticks(range(0, max_mode+1, 5))
ax1.yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.1f'))
ax1.set_xlim([0, max_mode-5])
ax1.set_xlabel(modes_xlabel)
ax1.set_ylabel(modes_ylabel)
ax1.legend([plot_labels[0],plot_labels[1]])
fig1.savefig(sys_params[0]["figure_location"]+"/init_modes" + sys_params[0]["plot_file_type"], dpi=300, bbox_inches='tight')

## Pressure modes

In [ ]:
hs_and_modes = [None] * num_comparisons
pabl_ave = [None] * num_comparisons
X_train_complex = [None] * num_comparisons
intensity_map_normalized = [None] * num_comparisons
power_spectrum = [None] * num_comparisons
rms2 = [None] * num_comparisons

LMAX = dataset_params[0]["LMAX"]

In [ ]:
for ind in range(num_comparisons):
    file_name = sys_params[ind]["heat_source_nc"]
    hs_and_modes[ind] = nrw.read_general_netcdf(run_location[ind]+"/"+file_name)

    if old_format:
        dataset_params[ind], facility_spec[ind] = tdg.define_dataset_params(int(num_examples))
    else:
        dataset_params[ind] = nrw.read_general_netcdf(diag_dir[ind] + "/" + sys_params[ind]["dataset_params_filename"])
        facility_spec[ind] = nrw.read_general_netcdf(diag_dir[ind] + "/" + sys_params[ind]["facility_spec_filename"])

    pabl_ave[ind] = hs_and_modes[ind]["average_flux"][0]
    
    X_train_complex[ind] = np_complex(hs_and_modes[ind]["complex_modes"][0,:], hs_and_modes[ind]["complex_modes"][1,:])

    intensity_map_normalized[ind] = hp.alm2map(X_train_complex[ind], LMAX)

    power_spectrum[ind] = uhp.alms2power_spectrum(X_train_complex[ind], LMAX)
    

In [ ]:
intensity_map0 = (intensity_map_normalized[0]+1)*pabl_ave[0]
intensity_map1 = (intensity_map_normalized[1]+1)*pabl_ave[1]

aspc = 1.2
fig1 = plt.figure(num=1, figsize=(6*aspc,6), facecolor='white')

nside = 30
npix = hp.nside2npix(nside)
#vec = hp.ang2vec(np.pi, np.pi * 3 / 4)
vec = hp.ang2vec(np.pi/2, np.pi * 1/2)
ipix_disc = hp.query_disc(nside=nside, vec=vec, radius=np.radians(90))
masked_intensity_map = intensity_map0
masked_intensity_map[ipix_disc] = intensity_map1[ipix_disc]

projection = hp.mollview(masked_intensity_map, unit=r"$\rm{Mbar}$", title="", flip="geo", return_projected_map=True)
hp.graticule()


In [ ]:

aspc = 2.0
fig2 = plt.figure(num=2, figsize=(6*aspc,6), facecolor='white')
ax2 = fig2.add_axes([0.15, 0.14, 0.6, 0.6])
cax1 = fig2.add_axes([0.76, 0.14, 0.02, 0.6])

cmesh = ax2.pcolormesh(projection, linewidth=0, rasterized=True)
cbar = fig2.colorbar(cmesh, ax=[ax2], cax=cax1)
ax2.axis('off')
ax2.plot([400.5,400.5],[0,400.0],"k", linewidth=1.0)

cbar.set_label("Pressure " + r"$(\rm{Mbar}$)")
fig2.savefig(sys_params[0]["figure_location"]+"/pressure_mollweide" + sys_params[0]["plot_file_type"], dpi=300, bbox_inches='tight')


In [ ]:
#aspc = 1.2
#fig2 = plt.figure(num=2, figsize=(6*aspc,3), facecolor='white')
fig2 = plt.figure(num=2, figsize=modes_aspect_ratio, facecolor='white')
ax2 = fig2.add_axes([0.15, 0.15, 0.8, 0.8])

for ind in range(num_comparisons):
    print(plot_labels[ind])
    ax2.plot(np.arange(LMAX), np.sqrt(power_spectrum[ind]) * 100.0, linewidth=3)
    rms2[ind] =np.sqrt(np.sum(np.array(power_spectrum[ind])))
    print('RMS is {:.2f}%, '.format(rms2[ind] * 100.0))

ax2.set_xticks(range(0, max_mode+1, 10))
ax2.set_xlim([0, max_mode-5])
ax2.set_xlabel(modes_xlabel)
ax2.set_ylabel(modes_ylabel)
#ax2.legend([plot_labels[0],plot_labels[1]])
fig2.savefig(sys_params[0]["figure_location"]+"/ablation_pressure_modes" + sys_params[0]["plot_file_type"], dpi=300, bbox_inches='tight')

## Absorbed power

In [ ]:
total_pwr = [None] * num_comparisons
surface_area = [None] * num_comparisons
avg_flux = [None] * num_comparisons
total_TW_cbet = [None] * num_comparisons

In [ ]:
for ind in range(num_comparisons):
    print(plot_labels[ind])
    
    theta_edges = (hs_and_modes[ind]["theta"][1:] + hs_and_modes[ind]["theta"][:-1]) / 2.0
    theta_edges = np.append(0.0, theta_edges)
    theta_edges = np.append(theta_edges, np.pi)
    phi_edges = (hs_and_modes[ind]["phi"][1:] + hs_and_modes[ind]["phi"][:-1]) / 2.0
    phi_edges = np.append(0.0, phi_edges)
    phi_edges = np.append(phi_edges, 2.0*np.pi)

    theta_grid, phi_grid = np.meshgrid(theta_edges, phi_edges)

    dphi = phi_grid[1:,1:] - phi_grid[:-1,1:]
    d_cos_theta = np.cos(theta_grid[1:,1:]) - np.cos(theta_grid[1:,:-1])
    domega = np.abs(dphi * d_cos_theta)

    total_pwr[ind] = np.sum(hs_and_modes[ind]["heat_source"] * domega)
    surface_area[ind] = 4.0 * np.pi * (facility_spec[ind]['target_radius'] / 10000.0)**2
    avg_flux[ind] = total_pwr[ind] / (4.0 * np.pi)
    total_TW_cbet[ind] = avg_flux[ind] * 1.0e-12 * (4.0 * np.pi)

    print('Mean intensity is {:.2e}W/cm^2, '.format(total_pwr[ind] / surface_area[ind])) #/ 4.0 / np.pi
    print('Intensity per steradian, {:.2e}W/sr'.format(avg_flux[ind]))
    print('The total power deposited is {:.2f}TW, '.format(total_TW_cbet[ind]))
    

## Calculation of fitness

In [ ]:
def fitness_function(rms1, rms2, pabl_ave):
    pabl_target = 50.0 #Mbars
    norm_factor = 10.0
    rms = np.sqrt((rms1 * 100.0)**2 / 9.0 +  (rms2 * 100.0)**2 / 18.0)

    maxi_func = np.exp(-rms) * (pabl_ave / pabl_target)**2 * norm_factor
    return maxi_func

"""
def fitness_function(rms1, rms2, pabl_ave):
    pabl_target = 65.0 #Mbars
    norm_factor = 1.0
    target_rms = 0.05
    rms = np.sqrt(rms1**2 + rms2**2)/2.0

    maxi_func = np.exp(-(rms / target_rms)**(1/2) + (pabl_ave / pabl_target)**2) * norm_factor
    return maxi_func
#"""

fitness = [None] * num_comparisons

In [ ]:

for ind in range(num_comparisons):
    print('For target: ', plot_labels[ind])
    print('intial RMS is {:.2f}%, '.format(rms1[ind] * 100.0))
    print(r'Average intensity {:.2e} W/cm^2, '.format(avg_flux[ind]))
    print('Ablation pressure RMS is {:.2f}%, '.format(rms2[ind] * 100.0))
    print('Average ablation pressure {:.2f}Mbar, '.format(pabl_ave[ind]))
    fitness[ind] = fitness_function(rms1[ind], rms2[ind], pabl_ave[ind])
    print('The fitness is {:.3f}, '.format(fitness[ind]) + "\n")


## Plot of combined illumination modes

In [ ]:
#aspc = 1.2
#fig1 = plt.figure(num=1, figsize=(6*aspc,3), facecolor='white')
fig1 = plt.figure(num=1, figsize=(3,6), facecolor='white')
ax1 = fig1.add_axes([0.15, 0.15, 0.8, 0.8])

for ind in range(num_comparisons):
    ax1.plot(np.arange(LMAX), power_spectrum_unweighted[ind] * 100.0 * 4.0 + np.sqrt(power_spectrum[ind]) * 100.0, linewidth=3)

ax1.set_xticks(range(0, max_mode+1, 5))
ax1.yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.1f'))
ax1.set_xlim([0, max_mode-5])
ax1.set_xlabel(modes_xlabel)
ax1.set_ylabel(modes_ylabel)
ax1.legend([plot_labels[0],plot_labels[1]])
#fig1.savefig(sys_params[0]["figure_location"]+"/init_modes" + sys_params[0]["plot_file_type"], dpi=300, bbox_inches='tight')

## Areal density modes

In [ ]:
from netCDF4 import Dataset
import numpy as np
from scipy.interpolate import griddata
import matplotlib.ticker as mticker
import copy

In [ ]:
dataset_dict = {}
#dataset_names = ["N190204_003_cbet", "Optimized_517_rescaled"]
#lot_labels = ["new","late CBET and bad rescaling"]
dataset_names = ["/mnt/local/dbarlow/solid_target/2305a_fixed_defocus/230601a_N190204_003_cbet/all_data",
                 #/mnt/local/dbarlow/solid_target/2305a_fixed_defocus/2307c_new_laser_pulses_for_hydro/time_varying_pulse_input_deck_rescaled",#]
                 "/mnt/local/dbarlow/solid_target/2305a_fixed_defocus/230828a_fixed_rescaling_start_wCBET"]
                
num_datasets = len(dataset_names)

plot_time = [9.0]#, 11.75] #[9.0, 11.75]
num_times = len(plot_time)
time_index_offset = [3]#, 6] #[3, 6]

In [ ]:
for dir_name in dataset_names:
    nc = Dataset(dir_name + "/" + 'dump31_1davg.nc')
    data = nc.groups['data']

    dataset_dict[dir_name] = {}

    dataset_dict[dir_name]["time"] = data.variables["time"][:]
    dataset_dict[dir_name]["x1c"]  = data.variables["x1c"][:,:]
    dataset_dict[dir_name]["dens"] = data.variables["dens"][:,:]
    dataset_dict[dir_name]["te"] = data.variables["Te"][:,:]
    dataset_dict[dir_name]["ti"] = data.variables["Ti"][:,:]
    dataset_dict[dir_name]["lm"] = data.variables["lm"][:,:]
    dataset_dict[dir_name]["sph"] = data.variables["sph"][:,:]
    dataset_dict[dir_name]["Pe"] = data.variables["Pe"][:,:]
    dataset_dict[dir_name]["Pi"] = data.variables["Pi"][:,:]
    dataset_dict[dir_name]["Z"] = data.variables["Z"][:,:]

    nc.close()

    dataset_dict[dir_name]["lm"][:,0] = 0.0
    dataset_dict[dir_name]["average_rhor"] = copy.copy(dataset_dict[dir_name]["sph"][:,0])
    dataset_dict[dir_name]["sph"][:,0] = 0.0

    dataset_dict[dir_name]["plot_time_index"] = np.array([0]*num_times)

    dataset_dict[dir_name]["P_tot"] = dataset_dict[dir_name]["Pi"] + dataset_dict[dir_name]["Pe"]
    print(np.shape(dataset_dict[dir_name]["P_tot"]))
    ind_max_ptot = int(np.argmax(dataset_dict[dir_name]["P_tot"])/np.shape(dataset_dict[dir_name]["P_tot"])[1])
    ind_max_dens = int(np.argmax(dataset_dict[dir_name]["dens"])/np.shape(dataset_dict[dir_name]["dens"])[1])
    print(ind_max_ptot, ind_max_dens)
    print("The highest pressure is at time: ", dataset_dict[dir_name]["time"][ind_max_ptot], "ns")
    print("The highest density is at time: ", dataset_dict[dir_name]["time"][ind_max_dens], "ns")
    
    for ind in range(num_times):
        ind_time = np.argmin(np.abs(dataset_dict[dir_name]["time"] - plot_time[ind]))
        if dir_name == dataset_names[-1]:
            ind_time = ind_time + time_index_offset[ind]
        dataset_dict[dir_name]["plot_time_index"][ind] = ind_time
        print(dataset_dict[dir_name]["time"][ind_time], ind_time)


In [ ]:
max_rhor = np.zeros((num_datasets))
colours = ["tab:blue", "tab:orange"]

for ind in range(num_times):
    
    #aspc = 1.2
    #fig3 = plt.figure(num=1, figsize=(6*aspc,3), facecolor='white')
    fig3 = plt.figure(num=3, figsize=modes_aspect_ratio, facecolor='white')
    fig4 = plt.figure()
    fig5 = plt.figure()
    fig3.clear();fig4.clear();fig5.clear()
    ax3 = fig3.add_axes([0.15, 0.15, 0.8, 0.8])
    ax4 = fig4.add_axes([0.15, 0.15, 0.8, 0.8])
    ax5 = fig5.add_axes([0.15, 0.15, 0.8, 0.8])
    counter = 0

    for dir_name in dataset_names:
        ind_rhor = dataset_dict[dir_name]["plot_time_index"][ind]
        print(ind_rhor)
        time = dataset_dict[dir_name]["time"]
        average_rhor = dataset_dict[dir_name]["average_rhor"]
        sph = dataset_dict[dir_name]["sph"]
        lm = dataset_dict[dir_name]["lm"]
        print("Plots created at time ", time[ind_rhor], "ns")

        ax3.plot(lm[ind_rhor,:], 100*sph[ind_rhor,:]/average_rhor[ind_rhor], linewidth=3, color=colours[counter])
        max_rhor[counter] = 100*max(sph[ind_rhor,:max_mode]/average_rhor[ind_rhor])
        ax3.plot(np.arange(LMAX), (power_spectrum_unweighted[counter] * 100.0 * 6.0 + np.sqrt(power_spectrum[counter]) * 100.0) / 35.0, linewidth=2, color=colours[counter], linestyle="dotted")

        rms =np.sqrt(np.sum(np.array((sph[ind_rhor,:LMAX]/average_rhor[ind_rhor])**2)))
        print('RMS is {:.4f}%, '.format(rms * 100.0))

        # plot pressure to check timing
        x1c = dataset_dict[dir_name]["x1c"]
        total_pressure = dataset_dict[dir_name]["P_tot"][ind_rhor,:] / 1.0e12# Convert to Mbar, Check!!
        ax4.plot(x1c[ind_rhor,:], total_pressure, linewidth=3)
        ax5.plot(x1c[ind_rhor,:], dataset_dict[dir_name]["dens"][ind_rhor,:], linewidth=3)
        counter += 1

    ax3.set_xticks(range(0, max_mode+1, 10))
    ax3.yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.1f'))
    #ax3.set_ylim(0, np.max(max_rhor)*1.1)
    ax3.set_xlim([0, max_mode-5])
    ax3.set_xlabel(modes_xlabel)
    ax3.set_ylabel(modes_ylabel)
    #ax3.legend([plot_labels[0],plot_labels[1]])
    
    xlim = 0.15 # 0.02
    ax4.set_xlim([0.0, xlim])
    ax5.set_xlim([0.0, xlim])

    fig3.savefig("plots/rms_rhoR_at_t_"+str(int(time[ind_rhor]))+"ns.pdf", dpi=300, bbox_inches='tight')
    fig4.savefig("plots/pressure_at_t_"+str(int(time[ind_rhor]))+"ns.pdf")
    

## Plot plasma profiles

In [ ]:
def critical_density(wavelength_l=351.0e-9):
    epi_0 = 8.85e-12
    mass_e = 9.11e-31
    charge_e = 1.6e-19
    c_s = 3.0e8
    
    omega_l = 2.0 * np.pi * c_s / wavelength_l
    
    n_crit = epi_0 * mass_e * omega_l**2 / charge_e**2
    
    print("Critical electron density for light wavelength {:.2f}nm is {:.2e}m^-3".format(wavelength_l*1.0e9,n_crit))
    
    return n_crit
    

In [ ]:
plot_time = [4.0]
num_times = len(plot_time)
time_index_offset = [0]

In [ ]:
n_crit = critical_density()
colours = ["tab:blue", "tab:orange"]

for ind in range(num_times):
    
    fig3 = plt.figure()
    fig4 = plt.figure()
    fig5 = plt.figure()
    fig3.clear();fig4.clear();fig5.clear()
    ax3 = fig3.add_axes([0.15, 0.15, 0.8, 0.8])
    ax4 = fig4.add_axes([0.15, 0.15, 0.8, 0.8])
    ax5 = fig5.add_axes([0.15, 0.15, 0.8, 0.8])
    
    counter = 0
    for dir_name in dataset_names:

        ind_time = np.argmin(np.abs(dataset_dict[dir_name]["time"] - plot_time[ind]))
        if dir_name == dataset_names[-1]:
            ind_time = ind_time + time_index_offset[ind]
        dataset_dict[dir_name]["plot_time_index"][ind] = ind_time
        print(dataset_dict[dir_name]["time"][ind_time], ind_time)

        time = dataset_dict[dir_name]["time"]
        print("Plots created at time ", time[ind_time], "ns")
        
        # plot pressure to check timing
        x1c = dataset_dict[dir_name]["x1c"]
        total_pressure = dataset_dict[dir_name]["P_tot"][ind_time,:] / 1.0e12# Convert to Mbar, Check!!
        ele_density_nc = dataset_dict[dir_name]["Z"][ind_time,:] * 1.0e6 / n_crit # Convert from cm to m and normalise to n_c
        ele_temp_keV = dataset_dict[dir_name]["te"][ind_time,:] / 1.0e3
        ion_temp_keV = dataset_dict[dir_name]["ti"][ind_time,:] / 1.0e3
        
        ax3.plot(x1c[ind_time,:], total_pressure, linewidth=3)
        ax4.semilogy(x1c[ind_time,:], ele_density_nc, linewidth=3)
        ax5.plot(x1c[ind_time,:], ele_temp_keV, linewidth=3, color=colours[counter])
        ax5.plot(x1c[ind_time,:], ion_temp_keV, linewidth=3, color=colours[counter], linestyle=":")
        counter += 1
    
    ind_start_comp = 850
    ind_finish_comp = 920
    print(dataset_dict[dir_name]["x1c"][ind_time,ind_start_comp], x1c[ind_time,ind_finish_comp])
    
    print(dataset_dict[dataset_names[0]]["te"][ind_time,ind_start_comp] / dataset_dict[dataset_names[1]]["te"][ind_time,ind_start_comp])
    
    xlim = [0.095, 0.35]
    ax3.set_xlim(xlim)
    ax3.set_ylim([0.0,50])
    ax4.set_xlim(xlim)
    ax4.set_ylim([0.01,2])
    ax5.set_xlim(xlim)
    
    ax3.set_xlabel("Radius (cm)")
    ax4.set_xlabel("Radius (cm)")
    ax5.set_xlabel("Radius (cm)")
    
    ax3.set_ylabel("Pressure (Mbar)")
    ax4.set_ylabel(r"Electron number density ($n_{crit}$)")
    ax5.set_ylabel("Temperature (keV)")
    

## Density plot and gated X-ray

In [ ]:
num_comparison = 2

path = [None] * num_comparison
filename = [None] * num_comparison
mesh_x = [None] * num_comparison
mesh_y = [None] * num_comparison
var_data = [None] * num_comparison
time = [None] * num_comparison
cmesh = [None] * num_comparison
id_xmin = [None] * num_comparison
id_xmax = [None] * num_comparison
id_ymin = [None] * num_comparison
id_ymax = [None] * num_comparison
#id_origin = [None] * num_comparison

path[0] = "/mnt/local/dbarlow/solid_target/2305a_fixed_defocus/230601a_N190204_003_cbet/all_data/2d_slices"
filename[0] = "dens2d_09001"#'dens2d_00058.img'
path[1] = "/mnt/local/dbarlow/solid_target/2305a_fixed_defocus/230828a_fixed_rescaling_start_wCBET/2d_slices"
#"/mnt/local/dbarlow/solid_target/2305a_fixed_defocus/2307c_new_laser_pulses_for_hydro/time_varying_pulse_input_deck_rescaled/2d_slices"
filename[1] = "dens2d_09301"#'dens2d_00058.img' 

radius_of_interest = 600.0


In [ ]:
for ind in range(num_comparison):
    with open(path[ind] + "/" + filename[ind] + ".bov") as f:
        for line in f:
            if "TIME" in line:
                time[ind] = int(float(line.split(" ")[-1][:-1]))
            if "VARIABLE" in line:
                var_name = line.split(" ")[1][:-1]
            if "DATA_SIZE" in line:
                #print(line.split(" "))
                x_ncells = int(line.split(" ")[1])
                y_ncells = int(line.split(" ")[2])
                print(x_ncells, y_ncells)
            if "BRICK_ORIGIN" in line:
                #print(line.split(" "))
                xmin = float(line.split(" ")[1])
                ymin = float(line.split(" ")[2])
                print(xmin, ymin)
            if "BRICK_SIZE" in line:
                #print(line.split(" "))
                xextent = float(line.split(" ")[2])
                yextent = float(line.split(" ")[4])
    xmax = xextent + xmin
    x = np.linspace(xmin, xmax, x_ncells)
    id_xmin[ind] = (np.abs(x + radius_of_interest)).argmin()
    id_xmax[ind] = (np.abs(x - radius_of_interest)).argmin()
    ymax = yextent + ymin
    y = np.linspace(ymin, ymax, y_ncells)
    id_ymin[ind] = (np.abs(y + radius_of_interest)).argmin()
    id_ymax[ind] = (np.abs(y - radius_of_interest)).argmin()

    mesh_x[ind], mesh_y[ind] = np.meshgrid(x, y)

    with open(path[ind] + "/" + filename[ind] + ".img", "rb") as f:
        # Seek backwards from end of file by 2 bytes per pixel 
        f.seek(-x_ncells*y_ncells*4, 2)
        var_data[ind] = np.fromfile(f, dtype=np.single).reshape((x_ncells, y_ncells))


In [ ]:
# Combine 2 datasets in one image

cut_ind = 400
combined_var_data = np.zeros((x_ncells, y_ncells))
combined_var_data[:,:cut_ind] = var_data[0][:,:cut_ind]
combined_var_data[:,cut_ind:] = var_data[1][:,cut_ind:]

combined_mesh_x = np.zeros((x_ncells, y_ncells))
combined_mesh_x[:,:cut_ind] = mesh_x[0][:,:cut_ind]
combined_mesh_x[:,cut_ind:] = mesh_x[1][:,cut_ind:]
combined_mesh_y = np.zeros((x_ncells, y_ncells))
combined_mesh_y[:,:cut_ind] = mesh_y[0][:,:cut_ind]
combined_mesh_y[:,cut_ind:] = mesh_y[1][:,cut_ind:]

aspc = 1.2
fig1 = plt.figure(num=1, figsize=(6*aspc,6), facecolor='white')
ax1 = fig1.add_axes([0.15, 0.15, 0.6, 0.6])
ax1.set_aspect('equal', adjustable='box')
cax1 = fig1.add_axes([0.73, 0.15, 0.05, 0.6])

cmesh = ax1.pcolormesh(combined_mesh_x, combined_mesh_y, combined_var_data, linewidth=0, rasterized=True)
ax1.plot([0,0],[np.min(mesh_y[ind]),np.max(mesh_y[ind])],"k", linewidth=0.5)
cbar = fig1.colorbar(cmesh, ax=[ax1], cax=cax1)

ax1.set_xlim([-radius_of_interest, radius_of_interest])
ax1.set_ylim([-radius_of_interest, radius_of_interest])

#circle2 = plt.Circle((0, 0), 330.0, color='w', fill=False)
#ax1.add_patch(circle2)

#ax1.text(-550,620.5, plot_labels[0], fontsize=22, color='black')
#ax1.text(20,620.5, plot_labels[1], fontsize=22, color='black')

ax1.set_xlabel(r"Distance $(\mu m)$")
ax1.set_ylabel(r"Distance $(\mu m)$")
cbar.set_label(var_name + r" $(g/cm^3)$")
fig1.savefig("plots/density_comparison_"+str(int(time[0]))+"ns.pdf", bbox_inches='tight')


## Pointings

In [ ]:
diag_dir = "Data_230630b_nif_cbet_defocus_10mm_new_fitness_rms_of_rms"
iex = 517
with_pointing_markers = True
import_flipped = False
old_format = False
display_steradians = False
sys_params = tdg.define_system_params(diag_dir)

In [ ]:
ind_profile = 0
dataset_params = nrw.read_general_netcdf(sys_params["root_dir"] + "/" + sys_params["dataset_params_filename"])
facility_spec = nrw.read_general_netcdf(sys_params["root_dir"] + "/" + sys_params["facility_spec_filename"])
dataset = nrw.read_general_netcdf(sys_params["root_dir"]+"/"+sys_params["trainingdata_filename"])

avg_flux = dataset["avg_flux"][iex, ind_profile]
real_modes = dataset["real_modes"][iex,ind_profile,:]
imag_modes = dataset["imag_modes"][iex,ind_profile,:]
rms = dataset["rms"][iex, ind_profile]
intensity_map_normalized = uhp.modes2imap(real_modes, imag_modes, dataset_params["imap_nside"])
intensity_map_sr = (intensity_map_normalized+1)*avg_flux

if ind_profile == 0:
    if display_steradians:
        drive_map = intensity_map_sr
        drive_units = r"$\rm{W/sr}$"
    else:
        drive_map = (intensity_map_normalized+1)*avg_flux / (facility_spec['target_radius'] / 10000.0)**2
        drive_units = r"$\rm{W/cm^2}$"
else:
    drive_map = intensity_map_sr
    drive_units = r"$\rm{Mbar}$"

In [ ]:
#hp.mollview(drive_map, unit=drive_units,flip="geo")
#hp.graticule()
aspc = 1.2
fig1 = plt.figure(num=1, figsize=(6*aspc,6), facecolor='white')
ax1 = fig1.add_axes([0.15, 0.14, 0.6, 0.6])

deck_gen_params = nrw.read_general_netcdf(sys_params["root_dir"] + "/" + sys_params["deck_gen_params_filename"])

port_theta = deck_gen_params["port_centre_theta"]
port_phi = deck_gen_params["port_centre_phi"]

pointing_theta = np.squeeze(deck_gen_params["theta_pointings"][iex,:])
pointing_phi = np.squeeze(deck_gen_params["phi_pointings"][iex,:])%(2.0 * np.pi)

num_ifriit_beams = int(facility_spec['nbeams'] / facility_spec['beams_per_ifriit_beam'])
port_cartesian = np.zeros((num_ifriit_beams, 2))
portc_cartesian = np.zeros((num_ifriit_beams, 2))
pointing_cartesian = np.zeros((num_ifriit_beams, 2))

for ibeam in range(0, num_ifriit_beams, 16):
    port_cartesian[ibeam,0], port_cartesian[ibeam,1] = uim.angle2moll(facility_spec["Theta"][ibeam], facility_spec["Phi"][ibeam])
    portc_cartesian[ibeam,0], portc_cartesian[ibeam,1] = uim.angle2moll(deck_gen_params["port_centre_theta"][ibeam], deck_gen_params["port_centre_phi"][ibeam])
    pointing_cartesian[ibeam,0], pointing_cartesian[ibeam,1] = uim.angle2moll(pointing_theta[ibeam], pointing_phi[ibeam])
    dx = pointing_phi[ibeam] - port_phi[ibeam]
    dy = pointing_theta[ibeam] - port_theta[ibeam]
    # reduce line length to give space for arrow head
    head_length = 0.05
    shaft_angle = np.arctan2(dy, dx)
    shaft_length = np.sqrt(dx**2 + dy**2) - head_length
    if shaft_length > 0.0:
        #dx = shaft_length * np.cos(shaft_angle)
        #dy = shaft_length * np.sin(shaft_angle)

        if np.abs(dx) < np.pi: # don't plot arrow for pointings which cross the edge
            head_length=0.0
            ax1.arrow(port_phi[ibeam], port_theta[ibeam], dx, dy, linewidth=1, head_width=head_length, head_length=head_length, fc='k', ec='k')
    else:
        x=1
        #ax1.arrow(port_theta[ibeam], port_phi[ibeam], dx, dy)

ax1.scatter(facility_spec["Phi"][:], facility_spec["Theta"][:], c="b", marker="s", s=20, edgecolors="w")
ax1.scatter(pointing_phi, pointing_theta, c="r", marker="x")

ax1.set_xlabel(r"$\phi$")
ax1.set_ylabel(r"$\theta$")

#plt.savefig(sys_params["figure_location"]+'/intensity_mollweide_'+str(iex) + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
hp.graticule()

for ibeam in range(0, num_ifriit_beams):
    port_cartesian[ibeam,0], port_cartesian[ibeam,1] = uim.angle2moll(facility_spec["Theta"][ibeam], facility_spec["Phi"][ibeam])
    portc_cartesian[ibeam,0], portc_cartesian[ibeam,1] = uim.angle2moll(deck_gen_params["port_centre_theta"][ibeam], deck_gen_params["port_centre_phi"][ibeam])
    pointing_cartesian[ibeam,0], pointing_cartesian[ibeam,1] = uim.angle2moll(pointing_theta[ibeam], pointing_phi[ibeam])
    
for ibeam in range(0, num_ifriit_beams, 16):
    dx = pointing_cartesian[ibeam,0] - portc_cartesian[ibeam,0]
    dy = pointing_cartesian[ibeam,1] - portc_cartesian[ibeam,1]
    # reduce line length to give space for arrow head
    head_length = 0.07
    shaft_angle = np.arctan2(dy, dx)
    shaft_length = np.sqrt(dx**2 + dy**2) - head_length
    if shaft_length > 0.0:
        dx = shaft_length * np.cos(shaft_angle)
        dy = shaft_length * np.sin(shaft_angle)

        if np.abs(dx) < np.pi: # don't plot arrow for pointings which cross the edge
            plt.arrow(portc_cartesian[ibeam,0], portc_cartesian[ibeam,1], dx, dy, linewidth=1, head_width=head_length, head_length=head_length, fc='k', ec='k')
    else:
        x=1
        #ax1.arrow(port_theta[ibeam], port_phi[ibeam], dx, dy)

plt.scatter(port_cartesian[:,0], port_cartesian[:,1], c="b", marker="s", s=20, edgecolors="w")
plt.scatter(pointing_cartesian[:,0], pointing_cartesian[:,1], c="r", marker="x")

plt.savefig(sys_params["figure_location"]+"/pointings_optimized" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')


## Import N190204-003 pointings

In [ ]:
import csv

In [ ]:
def pointings_pdd():   
    # opening the CSV file
    with open('ifriit_inputs_beams.csv', mode ='r') as csvfile:
        # reading the CSV file
        data = csv.reader(csvfile)

        names = []
        x = []
        y = []
        z = []
        # displaying the contents of the CSV file
        for lines in data:
            names.append(lines[0])
            x.append(np.double(lines[1]))
            y.append(np.double(lines[2]))
            z.append(np.double(lines[3]))

    x=np.array(x)
    y=np.array(y)
    z=np.array(z)
    names = np.array(names)

    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z/r)
    phi = np.arctan2(y,x)
    return names, x, y, z

In [ ]:
names,x,y,z = pointings_pdd()

In [ ]:
r = np.sqrt(x**2 + y**2 + z**2)
theta = np.arccos(z/r)
phi = np.arctan2(y,x)

r2 = r
r2 = 1100.0
sort_key = np.argsort(theta)
theta2 = theta[sort_key]
phi2 = phi[sort_key]
names2 = names[sort_key]

nbeams = (  16,    16,   16,   16,   32,   32,    16,    16,    16,    16)
angles = (23.5, 35.00, 46.0, 69.0, 83.0, 97.0, 111.0, 134.0, 145.0, 156.5)

i=0
for j in range(10):
    theta2[i:i+nbeams[j]] = angles[j]/180.0*np.pi
    i=i+nbeams[j]

fix_phi_names1 = ["B432","B432","B434","B434","B123","B123","B121","B121","B438","B438","B436","B436","B125","B125","B127","B127"]
fix_phi_names2 = ["B463","B464","B163","B164","B161","B162","B111","B112","B467","B468","B417","B418","B415","B416","B115","B116"]

for j in range(16):
    a = [i for i,item in enumerate(names2) if fix_phi_names1[j] in item]
    b = [i for i,item in enumerate(names2) if fix_phi_names2[j] in item]
    phi2[b] = phi2[a]

x2 = r2 * np.cos(phi2) * np.sin(theta2)
y2 = r2 * np.sin(phi2) * np.sin(theta2)
z2 = r2 * np.cos(theta2)

fig2 = plt.figure()
plt.plot(theta/np.pi*180.0,phi/np.pi*180.0,"*")
plt.plot(theta2/np.pi*180.0,phi2/np.pi*180.0,"r*")
plt.xlabel('$\\theta$')
plt.ylabel('$\phi$')

#for i in range(192):
#    plt.text(theta[i]/np.pi*180.0,phi[i]/np.pi*180.0,names[i],color="blue")
#    plt.text(theta2[i]/np.pi*180.0,phi2[i]/np.pi*180.0,names2[i],color="red",horizontalalignment='right')

#plt.show()

In [ ]:
hp.graticule()

#print(facility_spec["Beam"], names)
#print(type(facility_spec["Beam"][0]), type(names[0]))

backlighter_beam_names = ["B415","B416","B163","B164","B161","B162","B417","B418"]

for ibeam in range(0, num_ifriit_beams):
    arg_beam = np.where(names == facility_spec["Beam"][ibeam])
    port_cartesian[ibeam,0], port_cartesian[ibeam,1] = uim.angle2moll(facility_spec["Theta"][ibeam], facility_spec["Phi"][ibeam])
    portc_cartesian[ibeam,0], portc_cartesian[ibeam,1] = uim.angle2moll(deck_gen_params["port_centre_theta"][ibeam], deck_gen_params["port_centre_phi"][ibeam])
    pointing_cartesian[ibeam,0], pointing_cartesian[ibeam,1] = uim.angle2moll(theta[arg_beam], phi[arg_beam])
    if facility_spec["Beam"][ibeam] in backlighter_beam_names:
        pointing_cartesian[ibeam,0] = None
        pointing_cartesian[ibeam,1] = None

plot_arrow_ind = np.array([])
counter = 0
for ind in range(0, 11):
    plot_arrow_ind = np.append(plot_arrow_ind, np.linspace(0,3,4)+(counter*16))
    counter += 1
plot_arrow_ind = np.array(plot_arrow_ind, dtype='int')
#print(plot_arrow_ind)

for ibeam in plot_arrow_ind:
    dx = pointing_cartesian[ibeam,0] - portc_cartesian[ibeam,0]
    dy = pointing_cartesian[ibeam,1] - portc_cartesian[ibeam,1]
    # reduce line length to give space for arrow head
    head_length = 0.07
    shaft_angle = np.arctan2(dy, dx)
    shaft_length = np.sqrt(dx**2 + dy**2) - head_length
    if shaft_length > 0.0:
        dx = shaft_length * np.cos(shaft_angle)
        dy = shaft_length * np.sin(shaft_angle)

        if np.abs(dx) < np.pi: # don't plot arrow for pointings which cross the edge
            #head_length=0.0
            plt.arrow(portc_cartesian[ibeam,0], portc_cartesian[ibeam,1], dx, dy, linewidth=1, head_width=head_length, head_length=head_length, fc='k', ec='k')
    else:
        x=1
        #ax1.arrow(port_theta[ibeam], port_phi[ibeam], dx, dy)

plt.scatter(port_cartesian[:,0], port_cartesian[:,1], c="b", marker="s", s=20, edgecolors="w")
plt.scatter(pointing_cartesian[:,0], pointing_cartesian[:,1], c="r", marker="x")

plt.savefig(sys_params["figure_location"]+"/pointings_N190204_003" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')


## Emitted power

In [ ]:
import csv
import utils_intensity_map as uim
import netcdf_read_write as nrw
import utils_deck_generation as idg
import os

In [ ]:
iex = 517
run_dir = "Data_230630b_nif_cbet_defocus_10mm_new_fitness_rms_of_rms"
num_examples = 0
sys_params = tdg.define_system_params(run_dir)

dataset, dataset_params, deck_gen_params, facility_spec = idg.load_data_dicts_from_file(sys_params)
power_slice = slice(dataset_params["num_variables_per_beam"]-1, dataset_params["num_input_params"], dataset_params["num_variables_per_beam"])
cone_multiplier = deck_gen_params["sim_params"][iex,power_slice]
print(cone_multiplier)

num_cones = int(facility_spec['num_cones']/2) # Symmetry in top bottom hemisphere


In [ ]:
file_dirname = "laser_pulse_data_pdd/N190204-003"
file_name = "N190204-003-999_LEH_requested_beam_powers"
file_type = ".csv"

num_lines = sum(1 for line in open(file_dirname + "/" + file_name + file_type))

beam_indices = [0] * facility_spec['nbeams']
beam_names = [0] * facility_spec['nbeams']
times = np.zeros((num_lines-1))
powers = np.zeros((facility_spec['nbeams'],num_lines-1))

total_power = 0

pulse_file = open(file_dirname + "/" + file_name + file_type)
pulse_data = csv.reader(pulse_file)

backlighter_beams = ("B161","B162","B163","B164","B415","B416","B417","B418")

irow=0
for row in pulse_data:
    if irow == 0:
        nbeams = len(row) - 1
        for ibeam in range(0,nbeams):
            #print(row[ibeam][2:])
            beam_names[ibeam] = "B"+row[ibeam+1][2:]
            beam_indices[ibeam] = np.where(facility_spec['Beam']==beam_names[ibeam])[0][0]
    else:
        times[irow-1] = row[0]
        powers[0:,irow-1] = row[1:]
    irow += 1
pulse_file.close()

backlighter_modified_beams = []
idealized_total_power_profile = 0
total_power_no_backlighter = 0

fig = plt.figure()
ax = plt.axes()
for ibeam in range(0,nbeams):
    if beam_names[ibeam] in backlighter_beams:
        print("Beam: " + beam_names[ibeam] + " is for the backlighter")
    else:
        total_power_no_backlighter += powers[ibeam,:]
        plt.plot(times, powers[ibeam,:])
    if np.max(powers[ibeam,:]) > 1.0:
        backlighter_modified_beams.append(beam_names[ibeam])
print(backlighter_modified_beams)
plt.xlim([-0.5, 5.0])
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
plt.savefig(file_dirname+'/laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

total_power_with_backlighter = np.sum(powers, axis=0)
total_energy_with_backlighter = np.trapz(total_power_with_backlighter, x=times)
print("Total energy requested (with backlighter): {:.2f}".format(total_energy_with_backlighter), "kJ")
print("Max power (with backlighter): {:.2f}".format(np.max(total_power_with_backlighter)), "TW")

total_power = total_power_no_backlighter
total_energy = np.trapz(total_power_no_backlighter, x=times)
print("Total energy requested: {:.2f}".format(total_energy), "kJ")
print("Max power: {:.2f}".format(np.max(total_power)), "TW")

fig = plt.figure()
ax = plt.axes()
plt.plot(times, total_power_no_backlighter)
plt.xlim([-0.5, 5.0])
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")
plt.savefig(file_dirname+'/laser_pulse_shape_total' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

idealized_power_profile = total_power / nbeams

fig = plt.figure()
ax = plt.axes()
plt.plot(times, idealized_power_profile)
plt.xlim([-0.5, 5.0])
plt.xlabel("Time (ns)")
plt.ylabel("Idealized Power Profile per beam, no backlighter (TW)")
plt.savefig(file_dirname+'/idealized_laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

times_old = times
powers_old = powers
total_power_old = total_power
times = np.zeros((num_cones,num_lines-1))
powers = np.zeros((num_cones,num_lines-1))
total_power = np.zeros((num_lines-1))

for icone in range(0,num_cones):
    powers[icone,:] = idealized_power_profile
    times[icone,:] = times_old
    total_power += powers[icone,:]

In [ ]:
normalized_power = total_power / np.max(total_power)
total_power_reweighted = 0.0
powers_reweighted = np.zeros((np.shape(times)))

for ibeam in range(facility_spec['nbeams']):
    beam = facility_spec['Beam'][ibeam]
    file_name = file_dirname + "/pulse_" + beam + ".txt"
    
    cone_name = facility_spec["Cone"][ibeam]
    if (cone_name == 23.5):
        icone = 0
    elif (cone_name == 30):
        icone = 1
    elif (cone_name == 44.5):
        icone = 2
    else:
        icone = 3
    powers_reweighted[icone,:] = normalized_power[:] * cone_multiplier[icone]
    total_power_reweighted += powers_reweighted[icone,:]
    
    #with open(file_name,"w") as out:
    #    for irow in range(len(times[0,:])):
    #        out.write(str(times[icone,irow]) + ' ' + str(powers_reweighted[icone,irow]))
    #        out.write('\r\n')
    #out.close()

print(np.max(powers_reweighted, axis=1))
fig = plt.figure()
ax = plt.axes()
for icone in range(num_cones):
    plt.plot(times[icone,:], powers_reweighted[icone,:], label=facility_spec['cone_names'][icone])
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
plt.xlim([-0.5, 6.0])
plt.legend()
plt.savefig(file_dirname+'/new_unscaled_laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

total_energy_reweighted = np.trapz(total_power_reweighted, x=times[0,:])
print("Total energy requested: {:.2f}".format(total_energy_reweighted), "kJ")
#total_power = powers[0,:] * 32 + powers[1,:] * 32 + powers[2,:] * 64 + powers[3,:] * 64
print("Max power: {:.2f}".format(np.max(total_power_reweighted)), "TW")

fig = plt.figure()
ax = plt.axes()
plt.plot(times[0,:], total_power_reweighted)
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")
plt.xlim([-0.5, 6.0])
plt.savefig(file_dirname+'/new_unscaled_laser_pulse_shape_total' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')


In [ ]:
scaling = total_energy / total_energy_reweighted
print(scaling)

normalized_power = total_power / np.max(total_power)
total_power_reweighted = 0.0
powers_reweighted = np.zeros((np.shape(times)))

for ibeam in range(facility_spec['nbeams']):
    beam = facility_spec['Beam'][ibeam]
    file_name = file_dirname + "/pulse_" + beam + ".txt"
    
    cone_name = facility_spec["Cone"][ibeam]
    if (cone_name == 23.5):
        icone = 0
    elif (cone_name == 30):
        icone = 1
    elif (cone_name == 44.5):
        icone = 2
    else:
        icone = 3
    powers_reweighted[icone,:] = normalized_power[:] * cone_multiplier[icone] * scaling
    total_power_reweighted += powers_reweighted[icone,:]
    
    with open(file_name,"w") as out:
        for irow in range(len(times[0,:])):
            out.write(str(times[icone,irow]) + ' ' + str(powers_reweighted[icone,irow]))
            out.write('\r\n')
    out.close()

print(np.max(powers_reweighted, axis=1))
fig = plt.figure()
ax = plt.axes()
for icone in range(num_cones):
    plt.plot(times[icone,:], powers_reweighted[icone,:], label=facility_spec['cone_names'][icone])
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
plt.legend()
plt.xlim([-0.5, 6.0])
plt.savefig(file_dirname+'/new_scaled_laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

total_energy_reweighted = np.trapz(total_power_reweighted, x=times[0,:])
print("Total energy requested: {:.2f}".format(total_energy_reweighted), "kJ")
#total_power = powers[0,:] * 32 + powers[1,:] * 32 + powers[2,:] * 64 + powers[3,:] * 64
max_power_emitted_requested = np.max(total_power_reweighted)
print("Max power: {:.2f}".format(max_power_emitted_requested), "TW")

fig = plt.figure()
ax = plt.axes()
plt.plot(times[0,:], total_power_reweighted)
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")
plt.xlim([-0.5, 6.0])
plt.savefig(file_dirname+'/new_scaled_laser_pulse_shape_total' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')


In [ ]:
file_name = "N190204-003-999_LEH_beam_powers"
file_type = ".csv"

num_lines = sum(1 for line in open(file_dirname + "/" + file_name + file_type))

beam_indices = [0] * facility_spec['nbeams']
beam_names = [0] * facility_spec['nbeams']
times = np.zeros((num_lines-1))
powers = np.zeros((facility_spec['nbeams'],num_lines-1))

total_power = 0

pulse_file = open(file_dirname + "/" + file_name + file_type)
pulse_data = csv.reader(pulse_file)

backlighter_beams = ("B161","B162","B163","B164","B415","B416","B417","B418")

irow=0
for row in pulse_data:
    if irow == 0:
        nbeams = len(row) - 1
        for ibeam in range(0,nbeams):
            #print(row[ibeam][2:])
            beam_names[ibeam] = "B"+row[ibeam+1][2:]
            beam_indices[ibeam] = np.where(facility_spec['Beam']==beam_names[ibeam])[0][0]
    else:
        times[irow-1] = row[0]
        powers[0:,irow-1] = row[1:]
    irow += 1
pulse_file.close()

backlighter_modified_beams = []
idealized_total_power_profile = 0
total_power_no_backlighter = 0

fig = plt.figure()
ax = plt.axes()
for ibeam in range(0,nbeams):
    if beam_names[ibeam] in backlighter_beams:
        print("Beam: " + beam_names[ibeam] + " is for the backlighter")
    else:
        total_power_no_backlighter += powers[ibeam,:]
        plt.plot(times, powers[ibeam,:])
    if np.max(powers[ibeam,:]) > 1.0:
        backlighter_modified_beams.append(beam_names[ibeam])
print(backlighter_modified_beams)
plt.xlim([-0.5, 5.0])
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
plt.savefig(file_dirname+'/laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

total_power_with_backlighter = np.sum(powers, axis=0)
total_energy_with_backlighter = np.trapz(total_power_with_backlighter, x=times)
print("Total energy requested (with backlighter): {:.2f}".format(total_energy_with_backlighter), "kJ")
print("Max power (with backlighter): {:.2f}".format(np.max(total_power_with_backlighter)), "TW")

total_power = total_power_no_backlighter
total_energy = np.trapz(total_power_no_backlighter, x=times)
print("Total energy requested: {:.2f}".format(total_energy), "kJ")
print("Max power: {:.2f}".format(np.max(total_power)), "TW")

fig = plt.figure()
ax = plt.axes()
plt.plot(times, total_power_no_backlighter)
plt.xlim([-0.5, 5.0])
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")
plt.savefig(file_dirname+'/laser_pulse_shape_total' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

idealized_power_profile = total_power / nbeams

fig = plt.figure()
ax = plt.axes()
plt.plot(times, idealized_power_profile)
plt.xlim([-0.5, 5.0])
plt.xlabel("Time (ns)")
plt.ylabel("Idealized Power Profile per beam, no backlighter (TW)")
plt.savefig(file_dirname+'/idealized_laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

times_old = times
powers_old = powers
total_power_old = total_power
times = np.zeros((num_cones,num_lines-1))
powers = np.zeros((num_cones,num_lines-1))
total_power = np.zeros((num_lines-1))

for icone in range(0,num_cones):
    powers[icone,:] = idealized_power_profile
    times[icone,:] = times_old
    total_power += powers[icone,:]

## Printout stats from unscaled laser pulse (rescale power fractions)

In [ ]:
#N190204_003_total_energy = 408.95 #kJ
#unscaled_total_energy = 308.59 #kJ

#scaling = N190204_003_total_energy / unscaled_total_energy
print(scaling)

In [ ]:
def extract_run_parameters(iex, dataset_params, facility_spec, sys_params, deck_gen_params, use_ablation_pressure=0):

    total_power = 0
    print_line = []
    beam_count = 0
    num_vars = dataset_params["num_variables_per_beam"]
    ibeam = 0
    
    N190204_003_total_energy = 408.95#kJ
    unscaled_total_energy = 308.59#kJ

    scaling = N190204_003_total_energy / unscaled_total_energy
    print(scaling)

    for icone in range(facility_spec['num_cones']):
        beams_per_cone = facility_spec['beams_per_cone'][icone]

        pointing_theta = deck_gen_params["theta_pointings"][iex,ibeam]
        pointing_phi = deck_gen_params["phi_pointings"][iex,ibeam]%(2.0 * np.pi)
        
        cone_phi_offset = (pointing_phi-deck_gen_params["port_centre_phi"][ibeam])
        
        cone_defocus = deck_gen_params["defocus"][iex,beam_count]
        cone_powers = deck_gen_params["p0"][iex,beam_count] * scaling

        total_power += cone_powers * beams_per_cone
        beam_count = beam_count + int(beams_per_cone / facility_spec["beams_per_ifriit_beam"])

        if icone < int(facility_spec['num_cones']/2):
            print_line.append("For cone " + str(icone+1) +
                  ": {:.2f}\N{DEGREE SIGN}, ".format(np.degrees(pointing_theta)) +
                  "{:.2f}\N{DEGREE SIGN}, ".format(np.degrees(cone_phi_offset)) +
                  "{:.2f}mm, ".format(cone_defocus) +
                  "{:.2f}% power, ".format(cone_powers * 100))
        ibeam += beams_per_cone

    return print_line

In [ ]:
diag_dir = "Data_230630b_nif_cbet_defocus_10mm_new_fitness_rms_of_rms"
iex = 517
with_pointing_markers = True
import_flipped = False
old_format = False
display_steradians = False
sys_params = tdg.define_system_params(diag_dir)

run_location = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(iex)
print_list1 = []
print_list2 = []
print_list3 = []

In [ ]:
ind_profile = 0
dataset_params = nrw.read_general_netcdf(sys_params["root_dir"] + "/" + sys_params["dataset_params_filename"])
facility_spec = nrw.read_general_netcdf(sys_params["root_dir"] + "/" + sys_params["facility_spec_filename"])
dataset = nrw.read_general_netcdf(sys_params["root_dir"]+"/"+sys_params["trainingdata_filename"])

avg_flux = dataset["avg_flux"][iex, ind_profile]
real_modes = dataset["real_modes"][iex,ind_profile,:]
imag_modes = dataset["imag_modes"][iex,ind_profile,:]
rms = dataset["rms"][iex, ind_profile]

intensity_map_normalized = uhp.modes2imap(real_modes, imag_modes, dataset_params["imap_nside"])
intensity_map_sr = (intensity_map_normalized+1)*avg_flux

if ind_profile == 0:
    if display_steradians:
        drive_map = intensity_map_sr
        drive_units = r"$\rm{W/sr}$"
    else:
        drive_map = (intensity_map_normalized+1)*avg_flux / (facility_spec['target_radius'] / 10000.0)**2
        drive_units = r"$\rm{W/cm^2}$"
else:
    drive_map = intensity_map_sr
    drive_units = r"$\rm{Mbar}$"

complex_modes = np_complex(real_modes, imag_modes)
power_spectrum = uhp.alms2power_spectrum(complex_modes, dataset_params["LMAX"])

In [ ]:
deck_gen_params = nrw.read_general_netcdf(sys_params["root_dir"] + "/" + sys_params["deck_gen_params_filename"])

#print_list1, power_deposited = uim.readout_intensity(facility_spec, intensity_map_sr, use_ablation_pressure=ind_profile)
print_list2 = extract_run_parameters(iex, dataset_params, facility_spec, sys_params, deck_gen_params, use_ablation_pressure=ind_profile)
print_list = print_list1 + print_list2
stats_filename = sys_params["figure_location"]+"/stats"+str(iex)+".txt"
uim.print_save_readout(print_list, stats_filename)

## Absorption fraction

In [ ]:
def strided_app(a, L, S ):  # Window len = L, Stride len/stepsize = S
    nrows = ((a.size-L)//S)+1
    n = a.strides[0]
    return np.lib.stride_tricks.as_strided(a, shape=(nrows,L), strides=(S*n,n))

In [ ]:
for ind in range(num_comparisons):
    print(plot_labels[ind])
    predicted_abs_fraction = total_TW_cbet[ind] / max_power_emitted_requested
    print("Predicted absorption fraction at peak power: {:.2f} %".format(predicted_abs_fraction * 100.0))

In [ ]:
num_comparison = 2
window_len = 3

path[0] = "/mnt/local/dbarlow/solid_target/2305a_fixed_defocus/230601a_N190204_003_cbet/all_data"
filename = "abs_beam_total.txt"
path[1] = "/mnt/local/dbarlow/solid_target/2305a_fixed_defocus/230828a_fixed_rescaling_start_wCBET"

times = [None] * num_comparison
incoming_powers = [None] * num_comparison
scatter_powers = [None] * num_comparison

aspc = 1.2
fig1 = plt.figure(num=1, figsize=(6*aspc,6), facecolor='white')
ax1 = fig1.add_axes([0.15, 0.14, 0.6, 0.6])
fig2 = plt.figure(num=2, figsize=(6*aspc,6), facecolor='white')
ax2 = fig2.add_axes([0.15, 0.14, 0.6, 0.6])
fig3 = plt.figure(num=3, figsize=(6*aspc,6), facecolor='white')
ax3 = fig3.add_axes([0.15, 0.14, 0.6, 0.6])

for ind in range(num_comparisons):
    print(plot_labels[ind])
    time = []
    incoming_power = []
    scatter_power = []
    
    with open(path[ind] + "/" + filename) as f:
        for line in f:
            line = line.split(" ")
            if "#" in line:
                titles = line
            else:
                time.append(float(line[2]))
                incoming_power.append(float(line[4]))
                scatter_power.append(float(line[6]))
    
    time = np.array(time)
    incoming_power = np.array(incoming_power)
    scatter_power = np.array(scatter_power)
    
    ind_duplicates = np.where(time[:-1] - time[1:] > 0.0)[0]
    #print(ind_duplicates)
    num_removed = 0
    orginal_len = len(time)
    for ind_d in ind_duplicates:
        #print(ind_d, ind_d - num_removed)
        ind_d = ind_d - num_removed
        #print(ind_d, time[ind_d-1], time[ind_d], time[ind_d+1])
        new_ind_d = np.argmin(np.abs(time[:ind_d] - time[ind_d+1]))
        #print(new_ind_d, time[new_ind_d-2], time[new_ind_d-1], time[new_ind_d], time[new_ind_d+1])
        time = np.concatenate((time[:new_ind_d-1],time[ind_d+1:]))
        incoming_power = np.concatenate((incoming_power[:new_ind_d-1],incoming_power[ind_d+1:]))
        scatter_power = np.concatenate((scatter_power[:new_ind_d-1],scatter_power[ind_d+1:]))
        num_removed = orginal_len - len(time)
        #print(num_removed)
        #print(new_ind_d-1, time[new_ind_d-2], time[new_ind_d-1], time[new_ind_d])
    
    scatter_power = np.median(strided_app(scatter_power, window_len,1),axis=1)
    
    times[ind] = time
    incoming_powers[ind] = incoming_power
    scatter_powers[ind] = scatter_power
    
    absorbed_powers = incoming_powers[ind][1:-1]-scatter_powers[ind]
    
    total_incoming_energy = np.trapz(incoming_powers[ind], x=times[ind] / 1.0e3)
    print("The incoming energy: {:.2f} kJ".format(total_incoming_energy))
    total_scatter_energy = np.trapz(scatter_powers[ind], x=times[ind][1:-1] / 1.0e3)
    print("The scattered energy: {:.2f} kJ".format(total_scatter_energy))
    total_absorbed_energy = np.trapz(absorbed_powers, x=times[ind][1:-1] / 1.0e3)
    print("The absorbed energy: {:.2f} kJ".format(total_absorbed_energy))
    total_absorbed_fraction_energy = total_absorbed_energy / total_incoming_energy
    print("Fraction of energy absorbed: {:.2f} %".format(total_absorbed_fraction_energy * 100.0))
    
    ax1.plot(times[ind] / 1.0e3, incoming_powers[ind], linewidth=3)
    ax2.plot(times[ind][1:-1] / 1.0e3, scatter_powers[ind], linewidth=3)
    ax3.plot(times[ind][1:-1] / 1.0e3, absorbed_powers, linewidth=3)

    
ax1.set_xlabel("Time (ns)")
ax1.set_ylabel("Incoming Power (TW)")
ax1.legend([plot_labels[0],plot_labels[1]])

ax2.set_xlabel("Time (ns)")
ax2.set_ylabel("Scattered Power (TW)")
ax2.set_ylim(0,50.0)

ax3.set_xlabel("Time (ns)")
ax3.set_ylabel("Absorbed Power (TW)")
ax3.set_ylim(0,150.0)

fig1.savefig("plots/laser_pulse_incoming_power.pdf", dpi=300, bbox_inches='tight')
fig2.savefig("plots/laser_pulse_scattered_power.pdf", dpi=300, bbox_inches='tight')
fig3.savefig("plots/laser_pulse_absorbed_power.pdf", dpi=300, bbox_inches='tight')